## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
import joblib
from xgboost import XGBClassifier
from pathlib import Path
import time
from datetime import datetime
import warnings
import xgboost as xgb
import pickle
import logging
import json
import smote

## User Inputs

In [2]:
# User Provided Parameters - Futures Configuration
SYMBOL = "ES"
K = 0.2

# # Suffix for all output files (e.g., '_v3'). Change as needed.
SUFFIX = "_0825"
VERSION = "_v10"


In [3]:
with open('user_parameters.json', 'r') as file:
    data = json.load(file)

asset_data = data['assets'][SYMBOL]

# Load values into individual variables
S_PER_POINT = asset_data['price_per_point']                    
MIN_PRICE_RES = asset_data['minimum_resolution_points']       
PRICE_PER_TICK = asset_data['price_per_tick']                 
MAX_PENALTY = asset_data['maximum_drawdown']                   
MAX_DRAWDOWN = asset_data['maximum_drawdown']                  
POINTS_TO_MAX_DRAWDOWN = asset_data['points_to_maximum_drawdown']

TARGET_PROFIT = asset_data['targer_profit']
STOP_LOSS = asset_data['stoploss']


print(asset_data)

{'symbol': 'ES', 'price_per_point': 50.0, 'minimum_resolution_points': 0.25, 'price_per_tick': 12.5, 'maximum_drawdown': 150.0, 'points_to_maximum_drawdown': 3.0, 'targer_profit': 100, 'stoploss': 150, 'description': 'E-mini S&P 500 Futures'}


In [4]:
DIRECTORIES = {
    'DATA_DIR': '../data',
    'OUTPUT_DIR': '../output',
    'CONFIG_DIR': '../config',
    'PREPROCESSED_FILE': 'data/ES_preprocessed_all_features_training_zeros_corrected.csv',
    'XGB_TRAINED_MODEL_FILE': f'./output/{SYMBOL}_XGB_model{SUFFIX}{VERSION}.pkl',
    'RF_TRAINED_MODEL_FILE': f'./output/{SYMBOL}_RF_model{SUFFIX}{VERSION}.pkl',
    'DOCUMENTATION_FILE': f'./output/{SYMBOL}_model_documentation{SUFFIX}{VERSION}.md',
    'SYMBOL_SCALER_FILE': 'data/scalers/ES_scaler.pkl'
}

scaler = joblib.load('./data/ES_scaler.pkl')


## Dev Input

In [31]:
with open('selected_features.json', 'r') as file:
    selected_features = json.load(file)

SELECTED_FEATURES = selected_features['selected_features']

print(SELECTED_FEATURES)

['Low', 'High', 'Close', 'Open', 'Low_lag1', 'High_lag1', 'Close_lag1', 'Open_lag1', 'RSI', 'MACD', 'MyWAZLTTrend', 'HABodyRangeRatio', 'MACD_lag1', 'MyWAZLTTrend_lag1', 'HABodyRangeRatio_lag1', 'HAColor_1', 'HAColor_2', 'HAColor_3', 'HAColor_lag1_1', 'HAColor_lag1_2', 'HAColor_lag1_3', 'HALongWick_1', 'HALongWick_2', 'HALongWick_3', 'HALongWick_lag1_1', 'HALongWick_lag1_2', 'HALongWick_lag1_3', 'HACloseToEMALong_1', 'HACloseToEMALong_2', 'HACloseToEMALong_3', 'HACloseToEMALong_lag1_1', 'HACloseToEMALong_lag1_2', 'HACloseToEMALong_lag1_3', 'HALowToEMALong_1', 'HALowToEMALong_2', 'HALowToEMALong_3', 'HALowToEMALong_lag1_1', 'HALowToEMALong_lag1_2', 'HALowToEMALong_lag1_3', 'HAHighToEMALong_1', 'HAHighToEMALong_2', 'HAHighToEMALong_3', 'HAHighToEMALong_lag1_1', 'HAHighToEMALong_lag1_2', 'HAHighToEMALong_lag1_3', 'HACloseToEMAShort_1', 'HACloseToEMAShort_2', 'HACloseToEMAShort_3', 'HACloseToEMAShort_lag1_1', 'HACloseToEMAShort_lag1_2', 'HACloseToEMAShort_lag1_3', 'HALowToEMAShort_1', 'HAL

## Data Validation and Loading

In [32]:
# Validate input files
def validate_files():
    file_path = Path(DIRECTORIES['PREPROCESSED_FILE'])
    
    if not file_path.exists():
        raise FileNotFoundError()
        
    print("Input file found.\n")

validate_files()

Input file found.



In [33]:
# Load and validate data
def load_data():
    dtypes = {col: 'float64' for col in pd.read_csv(DIRECTORIES['PREPROCESSED_FILE'], nrows=0).columns[1:]}
    dtypes['Date'] = 'str'
    df = pd.read_csv(DIRECTORIES['PREPROCESSED_FILE'], dtype=dtypes)
    print(f"## Data Validation\nLoaded {len(df)} rows from {DIRECTORIES['PREPROCESSED_FILE']}\n")
    
    n_cols = len(df.columns)
    
    if n_cols == 333:
        print(f"Total column count: {n_cols}\n")
    else:
        error_msg = f"Expected 333 columns, found {n_cols}"
        raise ValueError(error_msg)
        
    date_formats = ['%Y-%m-%d %H:%M:%S', '%m/%d/%Y %I:%M:%S %p', '%m/%d/%y %I:%M:%S %p']
    parsed_dates = None
    for fmt in date_formats:
        try:
            parsed_dates = pd.to_datetime(df['Date'], format=fmt, errors='coerce')
            if parsed_dates.notna().any():
                print(f"Date parsing succeeded with format: {fmt}\n")
                break
        except Exception:
            continue
    
    if parsed_dates is None or parsed_dates.isna().all():
        try:
            parsed_dates = pd.to_datetime(df['Date'], format='mixed', errors='coerce')
            print("Date parsing succeeded with mixed format inference.\n")
        except Exception as e:
            error_msg = f"Failed to parse datetime: {str(e)}"
            print(error_msg)
            raise ValueError(error_msg)
    
    df['Date'] = parsed_dates
    invalid_dates = df['Date'].isna().sum()
    if invalid_dates > 0:
        print(f"Dropped {invalid_dates} rows with invalid dates.\n")
        df = df.dropna(subset=['Date'])
    
    print(f"Remaining rows after date validation: {len(df)}\n")
    
    seconds = df['Date'].dt.second
    invalid_precision = (seconds % 5 != 0).sum()
    if invalid_precision > 0:
        print(f"Found {invalid_precision} rows with non-5-second precision; dropping.\n")
        df = df[seconds % 5 == 0]
    
    print(f"Remaining rows after precision check: {len(df)}\n")
    
    df['Time'] = df['Date'].dt.time
    df['InWindow'] = df['Time'].apply(lambda x: pd.Timestamp('08:35:00').time() <= x <= pd.Timestamp('10:25:00').time())
    print(f"Bars in trading window (08:35–10:25): {df['InWindow'].sum()}\n")
    
    return df

# CALL TO LOAD DATA 
df = load_data()


## Data Validation
Loaded 94757 rows from data/ES_preprocessed_all_features_training_zeros_corrected.csv

Total column count: 333

Date parsing succeeded with format: %Y-%m-%d %H:%M:%S

Remaining rows after date validation: 94757

Remaining rows after precision check: 94757

Bars in trading window (08:35–10:25): 94757



## Preprocessing

In [34]:
# Calculate Base Profit from already descaled data
def calculate_profits(df):
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Calculating Base Profit from descaled data...")
    print(df.shape)
    # Since data is already descaled, we can use it directly
    data = df.iloc[:, 1:].values  # Exclude Date
    
    long_profits = np.zeros(len(df))
    short_profits = np.zeros(len(df))
    max_long_profits = np.zeros(len(df))
    max_short_profits = np.zeros(len(df))
    df['TradingDay'] = df['Date'].dt.date
    valid_indices = []
    
    for i in range(len(df) - 6):
        if not df['InWindow'].iloc[i]:
            continue
        current_day = df['TradingDay'].iloc[i]
        if i + 6 < len(df) and df['TradingDay'].iloc[i + 1:i + 7].eq(current_day).all():
            valid_indices.append(i)
            open_t1 = data[i + 1, df.columns.get_loc('Open') - 1]
            
            # Dynamic exit logic - check each bar for stop loss/target profit
            long_exit_bar = 6  # Default to hard exit at 6th bar
            short_exit_bar = 6  # Default to hard exit at 6th bar
            
            # Check for early exits due to stop loss or target profit
            for j in range(1, 7):
                if i + j >= len(df):
                    break
                    
                close_tj = data[i + j, df.columns.get_loc('Close') - 1]
                low_tj = data[i + j, df.columns.get_loc('Low') - 1]
                high_tj = data[i + j, df.columns.get_loc('High') - 1]
                
                # Check long position drawdown (stop loss)
                long_drawdown = ((open_t1 - low_tj) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                if long_drawdown >= STOP_LOSS and long_exit_bar == 6:
                    long_exit_bar = j
                
                # Check long position profit (target profit)
                long_profit_at_close = ((close_tj - open_t1) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                if long_profit_at_close >= TARGET_PROFIT and long_exit_bar == 6:
                    long_exit_bar = j
                
                # Check short position drawdown (stop loss)
                short_drawdown = ((high_tj - open_t1) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                if short_drawdown >= STOP_LOSS and short_exit_bar == 6:
                    short_exit_bar = j
                
                # Check short position profit (target profit)
                short_profit_at_close = ((open_t1 - close_tj) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                if short_profit_at_close >= TARGET_PROFIT and short_exit_bar == 6:
                    short_exit_bar = j
            
            # Calculate final profits based on exit bars
            close_long_exit = data[i + long_exit_bar, df.columns.get_loc('Close') - 1]
            close_short_exit = data[i + short_exit_bar, df.columns.get_loc('Close') - 1]
            
            # Long profit calculation
            long_points = (close_long_exit - open_t1) / MIN_PRICE_RES
            long_profit = long_points * (S_PER_POINT * MIN_PRICE_RES)
            lows_to_exit = data[i + 1:i + long_exit_bar + 1, df.columns.get_loc('Low') - 1]
            long_drawdown = ((open_t1 - min(lows_to_exit)) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
            long_profits[i] = -STOP_LOSS if long_drawdown >= STOP_LOSS else min(long_profit, TARGET_PROFIT)
            
            # Short profit calculation
            short_points = (open_t1 - close_short_exit) / MIN_PRICE_RES
            short_profit = short_points * (S_PER_POINT * MIN_PRICE_RES)
            highs_to_exit = data[i + 1:i + short_exit_bar + 1, df.columns.get_loc('High') - 1]
            short_drawdown = ((max(highs_to_exit) - open_t1) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
            short_profits[i] = -STOP_LOSS if short_drawdown >= STOP_LOSS else min(short_profit, TARGET_PROFIT)
            
            # Maximum profits (unchanged logic)
            lows_t1_t6 = data[i + 1:i + 7, df.columns.get_loc('Low') - 1]
            highs_t1_t6 = data[i + 1:i + 7, df.columns.get_loc('High') - 1]
            
            # Maximum Long Profit
            if long_drawdown >= STOP_LOSS:
                drawdown_bar = np.argmin(lows_to_exit) + 1
                max_long_profit = ((max(highs_t1_t6[:drawdown_bar]) - open_t1) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                max_long_profits[i] = max_long_profit
            else:
                max_long_profit = ((max(highs_t1_t6) - open_t1) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                max_long_profits[i] = max_long_profit
            
            # Maximum Short Profit
            if short_drawdown >= STOP_LOSS:
                drawdown_bar = np.argmax(highs_to_exit) + 1
                max_short_profit = ((open_t1 - min(lows_t1_t6[:drawdown_bar])) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                max_short_profits[i] = max_short_profit
            else:
                max_short_profit = ((open_t1 - min(lows_t1_t6)) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                max_short_profits[i] = max_short_profit
    
    valid_indices = np.array(valid_indices)
    print(f"Valid Profit Indices: {len(valid_indices)} bars with non-zero profits\n")
    
    return long_profits, short_profits, valid_indices

"""
Profit Calculation from Descaled Data
Computes long and short profits over a 6-bar horizon using descaled price data. 
Calculates base profits and maximum profits, applying a $150 drawdown penalty if exceeded. 
Returns arrays of long profits, short profits, and valid indices.
"""

# Calculate profits from descaled data
long_profits, short_profits, valid_indices = calculate_profits(df)




2025-08-04 22:54:45: Calculating Base Profit from descaled data...
(94757, 335)
Valid Profit Indices: 94325 bars with non-zero profits



In [35]:
# Scale continuous columns in the dataframe
def scale_continuous_columns(df, scaler):
    """Scale the continuous columns in the dataframe using the provided scaler"""
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Scaling continuous columns...")
    
    continuous_cols = [
        'Open', 'High', 'Low', 'Close', 'RSI', 'MACD', 'HABodyRangeRatio', 'MyWAZLTTrend',
        'Open_lag1', 'High_lag1', 'Low_lag1', 'Close_lag1', 'RSI_lag1', 'MACD_lag1', 'HABodyRangeRatio_lag1', 'MyWAZLTTrend_lag1'
    ]
    # Only keep columns that exist in df
    continuous_cols = [col for col in continuous_cols if col in df.columns]
    
    if not continuous_cols or scaler is None:
        print("Warning: No continuous columns to scale or scaler is None\n")
        return df
    
    # Create a copy of the dataframe
    df_scaled = df.copy()

    print(df_scaled.head())
    
    # Scale the continuous columns
    df_scaled[continuous_cols] = scaler.transform(df[continuous_cols])
    
    print(f"Continuous columns scaled successfully: {', '.join(continuous_cols)}\n")
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Continuous columns scaled successfully.")
    
    return df_scaled

"""
Scale Continuous Columns
Applies the provided `StandardScaler` to normalize continuous columns (e.g., Open, High, Low, Close, RSI) in the DataFrame. 
Creates a copy of the DataFrame to avoid modifying the original data. Logs the scaled columns for verification. 
Returns the scaled DataFrame for model training.
"""

# Scale the continuous columns for training
df_scaled = scale_continuous_columns(df, scaler)

"""
Scaling Verification
Verifies that the scaling of continuous columns (e.g., Open, High, Low, Close, RSI) was successful. 
Logs the mean and standard deviation of these columns before and after scaling. 
Ensures that post-scaling, the mean is approximately 0 and the standard deviation is approximately 1. 
Writes results to the documentation file for transparency.
"""

# Verify scaling worked correctly
continuous_cols = ['Open', 'High', 'Low', 'Close', 'RSI', 'MACD', 'HABodyRangeRatio', 'MyWAZLTTrend']
continuous_cols = [col for col in continuous_cols if col in df.columns]

print("## Scaling Verification\n")
print("Before scaling (descaled data):\n")
for col in continuous_cols:
    col_mean = df[col].mean()
    col_std = df[col].std()
    print(f"{col}: mean={col_mean:.4f}, std={col_std:.4f}\n")

print("After scaling:\n")
for col in continuous_cols:
    col_mean = df_scaled[col].mean()
    col_std = df_scaled[col].std()
    print(f"{col}: mean={col_mean:.4f}, std={col_std:.4f}\n")

print("Note: After scaling, continuous columns should have mean≈0 and std≈1\n")


2025-08-04 22:55:02: Scaling continuous columns...
                 Date     Open     High      Low   Close    RSI  MACD  \
0 2025-04-02 08:35:00  5683.75  5684.25  5683.50  5684.0  65.68  2.23   
1 2025-04-02 08:35:05  5684.25  5684.25  5682.25  5683.5  59.69  2.12   
2 2025-04-02 08:35:10  5683.50  5684.00  5682.50  5683.5  56.01  2.01   
3 2025-04-02 08:35:15  5683.50  5683.75  5681.50  5682.0  44.06  1.78   
4 2025-04-02 08:35:20  5681.75  5682.75  5681.25  5682.0  34.44  1.58   

   HABodyRangeRatio  MyWAZLTTrend  Open_lag1  ...  HACloseToEMAShort_lag1_3  \
0              0.25         0.931       0.00  ...                       0.0   
1              0.13         0.918    5683.75  ...                       0.0   
2              0.05         0.907    5684.25  ...                       0.0   
3              0.11         0.895    5683.50  ...                       0.0   
4              0.28         0.880    5683.50  ...                       1.0   

   HALowToEMAShort_1  HALowToEMASho

In [36]:
def split_data(df):
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Splitting data by custom cutoff and trading day...")

    # Ensure TradingDay column exists
    df['TradingDay'] = df['Date'].dt.date

    # Define test set cutoff
    test_start_date = pd.Timestamp('2025-07-07 08:30:00')
    
    # Split into train+val and test
    train_val_df = df[df['Date'] < test_start_date]
    test_df = df[df['Date'] >= test_start_date]
    
    # Split train+val by trading day
    trading_days = sorted(train_val_df['TradingDay'].unique())
    n_days = len(trading_days)
    train_days = trading_days[:int(0.8 * n_days)]
    val_days = trading_days[int(0.8 * n_days):]

    
    train_df = train_val_df[train_val_df['TradingDay'].isin(train_days)]
    val_df = train_val_df[train_val_df['TradingDay'].isin(val_days)]
    
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Data split completed:")
    print(test_df.head())
    print(f"  → Train: {len(train_df)} rows")
    print(f"  → Validation: {len(val_df)} rows")
    print(f"  → Test: {len(test_df)} rows starting from {test_df['Date'].iloc[0] if not test_df.empty else 'N/A'}")
    
    return train_df, val_df, test_df

"""
Data Splitting
Splits the scaled DataFrame into training (70%), validation (20%), and test (10%) sets based on trading days. 
Ensures temporal consistency by assigning entire trading days to each set.
Returns the training, validation, and test DataFrames.
"""

train_df, val_df, test_df = split_data(df_scaled)


2025-08-04 22:55:03: Splitting data by custom cutoff and trading day...
2025-08-04 22:55:03: Data split completed:
                     Date      Open      High       Low     Close       RSI  \
88152 2025-07-07 08:35:00  1.371027  1.372317  1.369823  1.370257  1.337226   
88153 2025-07-07 08:35:05  1.371027  1.373880  1.370603  1.373380  1.490997   
88154 2025-07-07 08:35:10  1.372589  1.373098  1.371383  1.371038  1.384822   
88155 2025-07-07 08:35:15  1.371027  1.371535  1.369043  1.371819  0.764703   
88156 2025-07-07 08:35:20  1.371808  1.371535  1.369043  1.371819  0.813214   

           MACD  HABodyRangeRatio  MyWAZLTTrend  Open_lag1  ...  \
88152 -0.360638          1.380425     -1.450387 -18.309853  ...   
88153 -0.219423          1.380425     -1.379309   1.371027  ...   
88154 -0.148816          1.278654     -1.335692   1.371027  ...   
88155 -0.078209          0.617138     -1.340538   1.372589  ...   
88156 -0.021723         -0.044378     -1.326000   1.371027  ...   

       

In [37]:
def select_important_features(df, n_features=76):
    """Select features directly from the static SELECTED_FEATURES list to ensure exact consistency."""
        
    found_features = []
    missing_features = []
    
    # Debug: Print the actual SELECTED_FEATURES list
    print(f"SELECTED_FEATURES length: {len(SELECTED_FEATURES)}")
    
    # Check which features from SELECTED_FEATURES exist in the dataframe
    for feat in SELECTED_FEATURES:
        if feat in df.columns:
            found_features.append(feat)
        else:
            missing_features.append(feat)
    
    print(f"Found {len(found_features)} features for model.")
    print(f"Missing features: {missing_features}")
    
    
    # Ensure we don't exceed the number of found features
    actual_features = found_features[:min(n_features, len(found_features))]
    
    print(f"Final selected features count: {len(actual_features)}\n")
    
    # Verify we have exactly the expected number of features
    if len(actual_features) != len(SELECTED_FEATURES):
        print(f"WARNING: Feature count mismatch! Expected {len(SELECTED_FEATURES)}, got {len(actual_features)}\n")
        if missing_features:
            print(f"Missing features that need to be added to dataset: {missing_features}\n")
    
    return actual_features

# Feature selection (76 features including price features)
important_features = select_important_features(df_scaled)

# Validate that we got exactly 76 features
if len(important_features) != 76:
    error_msg = f"ERROR: Expected 76 features, but got {len(important_features)}"
    raise ValueError(error_msg)

SELECTED_FEATURES length: 76
Found 76 features for model.
Missing features: []
Final selected features count: 76



In [38]:
# Get column indices for the selected features
feature_columns = [df_scaled.columns.get_loc(f) for f in important_features if f in df_scaled.columns]

print(f"Feature Validation: Successfully selected {len(important_features)} features\n")

# Get all features first, then select important ones
all_feature_data_train = train_df.iloc[:, 1:333].values
all_feature_data_val = val_df.iloc[:, 1:333].values
all_feature_data_test = test_df.iloc[:, 1:333].values

# Select only important features
feature_data_train = all_feature_data_train[:, [i-1 for i in feature_columns]]
feature_data_val = all_feature_data_val[:, [i-1 for i in feature_columns]]
feature_data_test = all_feature_data_test[:, [i-1 for i in feature_columns]]

print(f"Successfully using {feature_data_train.shape[1]} selected features (including price features)\n")
print(f"Data Shapes - Train: {feature_data_train.shape}, Val: {feature_data_val.shape}, Test: {feature_data_test.shape}\n")

train_indices = train_df.index.values
val_indices = val_df.index.values
test_indices = test_df.index.values #7260

# Filter indices to valid profits
train_valid_mask = np.isin(train_indices, valid_indices)
train_valid_indices = train_indices[train_valid_mask]
feature_data_train = feature_data_train[train_valid_mask]
train_df_filtered = train_df.iloc[train_valid_mask]

val_valid_mask = np.isin(val_indices, valid_indices)
val_valid_indices = val_indices[val_valid_mask]
feature_data_val = feature_data_val[val_valid_mask]
val_df_filtered = val_df.iloc[val_valid_mask]

test_valid_mask = np.isin(test_indices, valid_indices)
test_valid_indices = test_indices[test_valid_mask]
feature_data_test = feature_data_test[test_valid_mask]
test_df_filtered = test_df.iloc[test_valid_mask]


Feature Validation: Successfully selected 76 features

Successfully using 76 selected features (including price features)

Data Shapes - Train: (69899, 76), Val: (18253, 76), Test: (6605, 76)



In [39]:
# Reward function
def reward(action, long_profit, short_profit):
    if action == 0:  # Long
        if long_profit > 30:
            return long_profit
        elif 10 < long_profit <= 30:
            return -MAX_PENALTY * np.exp(-K * (long_profit - 10))
        else:
            return -MAX_PENALTY
    elif action == 1:  # Short
        if short_profit > 30:
            return short_profit
        elif 10 < short_profit <= 30:
            return -MAX_PENALTY * np.exp(-K * (short_profit - 10))
        else:
            return -MAX_PENALTY
    else:  # No Trade
        return 0

In [106]:
min_profit_threshold = 30   # Increased from 30

long_ratio = 0.1 # should select top 10% trades from all available long that has highest profit

short_ratio = 0.1 # should select top 10% trades from all available shorts that has highest profit

# Generate action labels with improved strategy
def generate_action_labels(long_profits, short_profits, indices, total_bars, is_test_set=False):

    print(f"Total bars: {total_bars}")
    
    labels = np.full(total_bars, 2)
    
    profit_df = pd.DataFrame({
        'index': indices,
        'long_profit': long_profits,
        'short_profit': short_profits,
        'long_reward': [reward(0, lp, sp) for lp, sp in zip(long_profits, short_profits)],
        'short_reward': [reward(1, lp, sp) for lp, sp in zip(long_profits, short_profits)]
    })
    
    # Calculate additional metrics for better selection
    profit_df['long_profit_ratio'] = np.where(profit_df['short_profit'] != 0, 
                                            profit_df['long_profit'] / np.abs(profit_df['short_profit']), 
                                            profit_df['long_profit'])
    profit_df['short_profit_ratio'] = np.where(profit_df['long_profit'] != 0, 
                                             profit_df['short_profit'] / np.abs(profit_df['long_profit']), 
                                             profit_df['short_profit'])
    
    # Enhanced selection criteria
    
    # Step 1: Select high-quality trades with multiple criteria
    long_candidates = profit_df[
        (profit_df['long_profit'] > min_profit_threshold) & 
        (profit_df['long_reward'] > 0) &
        (profit_df['long_profit'] > profit_df['short_profit'])
    ].copy()
    
    short_candidates = profit_df[
        (profit_df['short_profit'] > min_profit_threshold) & 
        (profit_df['short_reward'] > 0) &
        (profit_df['short_profit'] > profit_df['long_profit'])
    ].copy()
    
    # Calculate composite scores for ranking
    long_candidates['composite_score'] = (
        long_candidates['long_profit'] * 0.4 +
        long_candidates['long_reward'] * 0.3 +
        long_candidates['long_profit_ratio'] * 0.3
    )
    
    short_candidates['composite_score'] = (
        short_candidates['short_profit'] * 0.4 +
        short_candidates['short_reward'] * 0.3 +
        short_candidates['short_profit_ratio'] * 0.3
    )
    
    available_long = len(long_candidates)
    available_short = len(short_candidates)

    print(f"available_long: {available_long}")
    print(f"available_short: {available_short}")

    # Calculate current trade ratios
    long_ratio_data = available_long / total_bars if total_bars > 0 else 0
    short_ratio_data = available_short / total_bars if total_bars > 0 else 0
    total_trade_ratio = (available_long + available_short) / total_bars if total_bars > 0 else 0

    print(f"long_ratio: {long_ratio}")
    print(f"short_ratio: {short_ratio}")

    print(f"total_trade_ratio: {total_trade_ratio}")
   
    target_long_trades should select top 10% with highest profit 
    target_short_trades should select top 10% highest profit

    make any chnaging required but dont ditort the structire
    
    
    # Select top trades based on composite score
    if len(long_candidates) > 0:
        final_long_indices = long_candidates.nlargest(target_long_trades, 'composite_score').index
        labels[final_long_indices] = 0  # Long
    
    if len(short_candidates) > 0:
        final_short_indices = short_candidates.nlargest(target_short_trades, 'composite_score').index
        labels[final_short_indices] = 1  # Short

    return labels

"""
Action Label Generation
Generates action labels (Long, Short, No Trade) for training, validation, and test sets using profit data. Applies the `generate_action_labels` function to assign labels based on a reward function and profit thresholds. Uses filtered indices to ensure valid profit calculations. Logs the process and distribution of labels for each set.
"""

# Generate action labels
print(f"Generating action labels...")
train_labels = generate_action_labels(long_profits[train_valid_indices], short_profits[train_valid_indices], train_valid_indices, len(train_df_filtered), is_test_set=False)
# val_labels = generate_action_labels(long_profits[val_valid_indices], short_profits[val_valid_indices], val_valid_indices, len(val_df_filtered), is_test_set=False)
# test_labels = generate_action_labels(long_profits[test_valid_indices], short_profits[test_valid_indices], test_valid_indices, len(test_df_filtered), is_test_set=True)

print(dict(zip(unique, counts)))


## Balancing Class

In [72]:
# """
# SMOTE Application for Class Balancing
# Applies SMOTE (Synthetic Minority Oversampling Technique) to balance the training dataset across three classes: Long (0), Short (1), and No Trade (2). SMOTE generates synthetic samples for minority classes to address imbalance, using 3 nearest neighbors (`k_neighbors=3`). Logs the number of samples before and after SMOTE, along with the resulting class distribution. If SMOTE fails, reverts to original data and logs the failure.

# Reason for Applying SMOTE: The dataset likely has an imbalanced class distribution, with "No Trade" being the majority class due to selective trade criteria (e.g., high-profit thresholds). SMOTE ensures better representation of Long and Short classes, improving model performance on minority classes.

# Logic: SMOTE creates synthetic samples by interpolating between existing minority class samples, preserving data characteristics. It is applied to all three classes if any are underrepresented, aiming for equal class counts in `train_labels_balanced`. The `random_state=42` ensures reproducibility, and `k_neighbors=3` controls the interpolation range.

# Reason for Multi-Class SMOTE: SMOTE works for three classes by treating each minority class independently, generating synthetic samples to balance the dataset. The code validates this by logging the resulting class counts (`Long`, `Short`, `No Trade`), ensuring they are roughly equal post-SMOTE.
# """

# print("Before Smote: ",np.unique(train_labels,return_counts = True))
# # Apply SMOTE for better class balancing
# print(f"Applying SMOTE for class balancing...")
# smote = SMOTE(random_state=42, k_neighbors=3)
# try:# remove smote
#     feature_data_train_balanced, train_labels_balanced = smote.fit_resample(feature_data_train, train_labels)
    
#     print(f"SMOTE Applied: {len(feature_data_train)} -> {len(feature_data_train_balanced)} samples\n")
#     print(f"Balanced Distribution: Long={np.sum(train_labels_balanced==0)}, Short={np.sum(train_labels_balanced==1)}, No Trade={np.sum(train_labels_balanced==2)}\n")
# except:
#     feature_data_train_balanced = feature_data_train
#     train_labels_balanced = train_labels
#     print("SMOTE failed, using original data\n")

In [73]:
print("Before Smote: ",np.unique(train_labels,return_counts = True))
feature_data_train_balanced = feature_data_train
train_labels_balanced = train_labels

# print(train_labels_balanced.countvalues())

Before Smote:  (array([0, 1, 2]), array([ 1260,  1223, 67098]))


## Model Training

In [74]:
def hyperparameter_tuning(X_train, y_train, X_val, y_val):
    
    """Perform hyperparameter tuning using grid search"""
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Starting hyperparameter tuning...")
    
    # Define parameter grid
    param_grid = {
        'max_depth': [1,2,4, 5, 6,7,8],
        'learning_rate': [0.01,0.03, 0.05, 0.1,0.3,0.5],
        'n_estimators': [150,500, 800, 1000,1500],
        'min_child_weight': [1, 3, 5,7,9],
        'subsample': [0.5,0.6,0.7,0.8, 0.9],
        'colsample_bytree': [0.5,0.6,0.7,0.8, 0.9]
    }
    
    best_score = 0
    best_params = None
    
    # Calculate class weights
    class_counts = np.bincount(y_train)
    total_samples = len(y_train)
    class_weights = {
        0: total_samples / (3 * class_counts[0]),  # Long
        1: total_samples / (3 * class_counts[1]),  # Short
        2: total_samples / (3 * class_counts[2])   # No Trade
    }
    sample_weights = np.array([class_weights[label] for label in y_train])
    
    # Grid search with reduced combinations for speed
    # Existing combinations preserved
    param_combinations = [
        {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 800, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8},
        {'max_depth': 4, 'learning_rate': 0.1, 'n_estimators': 500, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.9},
        {'max_depth': 6, 'learning_rate': 0.03, 'n_estimators': 1000, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8},
        {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 1000, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.8},
        {'max_depth': 4, 'learning_rate': 0.05, 'n_estimators': 800, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.9},

        # New unique combinations added below:
        {'max_depth': 7, 'learning_rate': 0.3, 'n_estimators': 1500, 'min_child_weight': 5, 'subsample': 0.7, 'colsample_bytree': 0.7},
        {'max_depth': 2, 'learning_rate': 0.01, 'n_estimators': 500, 'min_child_weight': 9, 'subsample': 0.6, 'colsample_bytree': 0.6},
        {'max_depth': 1, 'learning_rate': 0.5, 'n_estimators': 800, 'min_child_weight': 7, 'subsample': 0.5, 'colsample_bytree': 0.5},
        {'max_depth': 8, 'learning_rate': 0.1, 'n_estimators': 150, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.6},
        {'max_depth': 6, 'learning_rate': 0.05, 'n_estimators': 1000, 'min_child_weight': 5, 'subsample': 0.7, 'colsample_bytree': 0.9}
    ]

    
    for i, params in enumerate(param_combinations):
        print(f"Testing combination {i+1}/{len(param_combinations)}: {params}")
        
        # Create model with current parameters
        model = XGBClassifier(
            objective='multi:softprob',
            num_class=3,
            eval_metric='mlogloss',
            gamma=0.1,
            reg_alpha=0.1,
            reg_lambda=1.0,
            random_state=42,
            **params
        )
        
        # Train model
        model.fit(
            X_train, y_train,
            sample_weight=sample_weights,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
        
        # Evaluate on validation set
        val_probs = model.predict_proba(X_val)
        
        # Custom prediction logic
        val_predictions = np.full(len(X_val), 2)
        for j in range(len(val_probs)):
            long_prob, short_prob, no_trade_prob = val_probs[j]
            if long_prob > 0.3 and long_prob > short_prob and long_prob > no_trade_prob:
                val_predictions[j] = 0
            elif short_prob > 0.3 and short_prob > long_prob and short_prob > no_trade_prob:
                val_predictions[j] = 1
        
        # Calculate accuracy
        accuracy = np.mean(val_predictions == y_val)
        
        # Calculate balanced accuracy (giving equal weight to each class)
        class_accuracies = []
        for class_label in [0, 1, 2]:
            class_mask = y_val == class_label
            if np.sum(class_mask) > 0:
                class_acc = np.mean(val_predictions[class_mask] == class_label)
                class_accuracies.append(class_acc)
        
        balanced_accuracy = np.mean(class_accuracies) if class_accuracies else 0
        
        if balanced_accuracy > best_score:
            best_score = balanced_accuracy
            best_params = params
    
    return best_params

In [75]:
def train_xgboost_model(X_train, y_train, X_val, y_val):
    """Train XGBoost model with updated parameters"""
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Training XGBoost model...")
    
    # Calculate class weights
    class_counts = np.bincount(y_train)
    total_samples = len(y_train)
    class_weights = {
        0: total_samples / (3 * class_counts[0]),  # Long
        1: total_samples / (3 * class_counts[1]),  # Short
        2: total_samples / (3 * class_counts[2])   # No Trade
    }
    
    # Updated XGBoost parameters
    xgb_params = {
        'objective': 'multi:softprob',
        'num_class': 3,
        'max_depth': 6,  # Increased depth
        'learning_rate': 0.05,  # Increased learning rate
        'n_estimators': 1000,  # More trees
        'min_child_weight': 3,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'gamma': 0.1,
        'reg_alpha': 0.1,
        'reg_lambda': 1,
        'scale_pos_weight': [class_weights[0], class_weights[1], class_weights[2]],  # Class weights
        'tree_method': 'hist',
        'random_state': 42
    }
    
    # Create and train model
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=50,
        verbose=100
    )
    
    return model

In [76]:
# """
# Hyperparameter Tuning
# Performs grid search to optimize XGBoost model hyperparameters using balanced training data. 
# Tests combinations of `max_depth`, `learning_rate`, `n_estimators`, and other parameters to maximize balanced accuracy on the validation set. 
# Logs each combination's performance and the best parameters found. Returns the optimal hyperparameters for model training.
# """

# # Hyperparameter tuning
# best_params = hyperparameter_tuning(feature_data_train_balanced, train_labels_balanced, feature_data_val, val_labels)



In [77]:
best_params = {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 1000, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.8}




In [78]:
"""
Ensemble Model Training
Trains XGBoost and Random Forest models on balanced training data for multi-class classification (Long, Short, No Trade). 
XGBoost uses optimized hyperparameters with `multi:softprob` objective, 
while Random Forest uses fixed parameters with balanced class weights. 
Saves both models to specified file paths for later use. Logs the training process and model file locations.
"""

# Train ensemble models
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Training ensemble models...")

# XGBoost model
xgb_model = XGBClassifier(
    objective='multi:softprob',
    num_class=3,
    eval_metric='mlogloss',
    gamma=0.1,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    **best_params
)
xgb_model.fit(
    feature_data_train_balanced, train_labels_balanced,
    eval_set=[(feature_data_val, val_labels)],
    verbose=False
)

# Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,#problem discuss with leafs, depth
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(feature_data_train_balanced, train_labels_balanced)

# Save models and feature information
joblib.dump(xgb_model, DIRECTORIES['XGB_TRAINED_MODEL_FILE'])
joblib.dump(rf_model, DIRECTORIES['RF_TRAINED_MODEL_FILE'])

2025-08-04 23:25:17: Training ensemble models...


['./output/ES_RF_model_0825_v10.pkl']

In [79]:
# Prediction Summary:
# Long     (0): 376
# Short    (1): 1559
# No Trade (2): 4670

In [80]:
#Smote + thresh = 35 + long 1 + short 0.7 == >> profit $325 (long = 400, short = -75)

## Model Evaluation

In [81]:
import numpy as np
import pandas as pd
from datetime import datetime
import os

def evaluate_model(xgb_model, rf_model, feature_data_test, test_labels, test_valid_indices, test_df, model_name):
    """Evaluate a model and log results with confusion matrices and detailed metrics"""
    
    # Get predictions from both models
    xgb_probs = xgb_model.predict_proba(feature_data_test)
    rf_probs = rf_model.predict_proba(feature_data_test)
    
    # Ensemble averaging with weights (XGB gets higher weight due to better performance)
    ensemble_probs = 0.7 * xgb_probs + 0.3 * rf_probs
    
    # Adaptive thresholding based on profit percentiles
    predicted_actions = np.full(len(feature_data_test), 2)  # Default to No Trade
    
    
    # Apply thresholds with profit considerations
    for i in range(len(ensemble_probs)):
        long_prob, short_prob, no_trade_prob = ensemble_probs[i]
        
        # Check if meets threshold and profit criteria
        test_idx = test_valid_indices[i] if i < len(test_valid_indices) else 0
        
        if (long_prob > short_prob and long_prob > no_trade_prob):  
            predicted_actions[i] = 0  # Long
        elif (short_prob >= long_prob and short_prob > no_trade_prob):
            predicted_actions[i] = 1  # Short
        # Otherwise remains No Trade (2)
    
    # ===== SAVE DETAILED RESULTS WITH DATES AND SIGNALS =====
    # Create detailed results DataFrame
    results_data = []
    signal_names = {0: 'Long', 1: 'Short', 2: 'No Trade'}
    
    # Get feature names if feature_data_test is a DataFrame, otherwise use generic names
    if hasattr(feature_data_test, 'columns'):
        feature_names = feature_data_test.columns.tolist()
    else:
        # If it's a numpy array, create generic feature names
        feature_names = [f'Feature_{i}' for i in range(feature_data_test.shape[1])]
    
    for i in range(len(feature_data_test)):
        test_idx = test_valid_indices[i] if i < len(test_valid_indices) else 0
        
        # Get date from test_df if available
        if 'Date' in test_df.columns:
            date = test_df.iloc[test_idx]['Date'] if test_idx < len(test_df) else None
        elif 'datetime' in test_df.columns:
            date = test_df.iloc[test_idx]['datetime'] if test_idx < len(test_df) else None
        else:
            date = None
            
        # Get probabilities
        long_prob, short_prob, no_trade_prob = ensemble_probs[i]
        
        # Get actual and predicted signals
        actual_signal = signal_names.get(test_labels[i], 'Unknown')
        predicted_signal = signal_names.get(predicted_actions[i], 'Unknown')
        
        # Get profit information
        long_profit = long_profits[test_idx] if test_idx < len(long_profits) else 0
        short_profit = short_profits[test_idx] if test_idx < len(short_profits) else 0
        
        # Determine which profit to use based on actual signal
        actual_profit = 0
        if test_labels[i] == 0:  # Long
            actual_profit = long_profit
        elif test_labels[i] == 1:  # Short
            actual_profit = short_profit
            
        # Calculate reward
        actual_reward = reward(test_labels[i], long_profit, short_profit) if 'reward' in globals() else 0
        
        # Create base record with all existing fields
        record = {
            'Date': date,
            'Index': test_idx,
            'Actual_Signal': actual_signal,
            'Predicted_Signal': predicted_signal,
            'Long_Probability': long_prob,
            'Short_Probability': short_prob,
            'NoTrade_Probability': no_trade_prob,
            'Long_Profit': long_profit,
            'Short_Profit': short_profit,
            'Actual_Profit': actual_profit,
            'Actual_Reward': actual_reward,
            'Correct_Prediction': test_labels[i] == predicted_actions[i],
            'Model_Name': model_name
        }
        
        # Add input features to the record
        if hasattr(feature_data_test, 'iloc'):
            # If it's a DataFrame, get features by column names
            for feature_name in feature_names:
                record[feature_name] = feature_data_test.iloc[i][feature_name]
        else:
            # If it's a numpy array, get features by index
            for j, feature_name in enumerate(feature_names):
                record[feature_name] = feature_data_test[i, j]
        
        results_data.append(record)
    
    # Create DataFrame and save to CSV
    results_df = pd.DataFrame(results_data)
    
    # Create output directory if it doesn't exist
    output_dir = 'evaluation_results'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{output_dir}/{model_name}_evaluation_results_{timestamp}.csv"
    
    # Save to CSV
    results_df.to_csv(filename, index=False)
    print(f"\n### Detailed Results Saved\n")
    print(f"Results saved to: {filename}\n")
    print(f"Total records saved: {len(results_df)}\n")
    
    # Print summary of saved data
    print(f"Signal Distribution in Saved Data:\n")
    print(f"Actual Signals: {results_df['Actual_Signal'].value_counts().to_dict()}\n")
    print(f"Predicted Signals: {results_df['Predicted_Signal'].value_counts().to_dict()}\n")
    print(f"Correct Predictions: {results_df['Correct_Prediction'].sum()}/{len(results_df)} ({results_df['Correct_Prediction'].mean()*100:.2f}%)\n")
    
    # ===== CONFUSION MATRIX ANALYSIS =====
    print(f"## {model_name} Detailed Evaluation\n")
    
    # Calculate confusion matrix
    confusion_mat = np.zeros((3, 3), dtype=int)
    for true, pred in zip(test_labels, predicted_actions):
        confusion_mat[true][pred] += 1
    
    # Print confusion matrix
    class_names = ['Long (0)', 'Short (1)', 'No Trade (2)']
    print("### Confusion Matrix:\n")
    print("Predicted →\n")
    print("Actual ↓    Long    Short    No Trade\n")
    for i, class_name in enumerate(class_names):
        print(f"{class_name:<10} {confusion_mat[i][0]:^8} {confusion_mat[i][1]:^8} {confusion_mat[i][2]:^10}\n")
    
    # Calculate detailed statistics
    print("\n### Detailed Statistics:\n")
    for true_class in range(3):
        total_actual = np.sum(test_labels == true_class)
        if total_actual == 0:
            continue
            
        correct = confusion_mat[true_class][true_class]
        accuracy = correct / total_actual if total_actual > 0 else 0
        
        print(f"\nActual {class_names[true_class]}:\n")
        print(f"Total Cases: {total_actual}\n")
        print(f"Correct Predictions: {correct} ({accuracy*100:.2f}%)\n")
        
        # Show misclassifications
        for pred_class in range(3):
            if pred_class != true_class:
                misclassified = confusion_mat[true_class][pred_class]
                if misclassified > 0:
                    print(f"Misclassified as {class_names[pred_class]}: {misclassified} ({misclassified/total_actual*100:.2f}%)\n")
    
    # Overall accuracy
    total_correct = np.sum(np.diag(confusion_mat))
    total_cases = np.sum(confusion_mat)
    overall_accuracy = total_correct / total_cases if total_cases > 0 else 0
    print(f"\nOverall Statistics:\n")
    print(f"Total Test Cases: {total_cases}\n")
    print(f"Total Correct Predictions: {total_correct} ({overall_accuracy*100:.2f}%)\n")
    print(f"Total Misclassifications: {total_cases - total_correct} ({(1-overall_accuracy)*100:.2f}%)\n")
    
    # ===== TRADING PERFORMANCE METRICS =====
    # Evaluation metrics
    actions = predicted_actions
    long_mask = (actions == 0) & test_df['InWindow'].values
    short_mask = (actions == 1) & test_df['InWindow'].values
    long_trades = np.sum(long_mask)
    short_trades = np.sum(short_mask)
    long_profits_list = [long_profits[test_valid_indices[i]] for i in range(len(actions)) if actions[i] == 0]
    short_profits_list = [short_profits[test_valid_indices[i]] for i in range(len(actions)) if actions[i] == 1]
    total_long_profit = sum(long_profits_list) if long_profits_list else 0
    total_short_profit = sum(short_profits_list) if short_profits_list else 0
    avg_long_profit = np.mean(long_profits_list) if long_profits_list else 0
    avg_short_profit = np.mean(short_profits_list) if short_profits_list else 0
    long_rewards = [reward(0, long_profits[test_valid_indices[i]], short_profits[test_valid_indices[i]]) for i in range(len(actions)) if actions[i] == 0]
    short_rewards = [reward(1, long_profits[test_valid_indices[i]], short_profits[test_valid_indices[i]]) for i in range(len(actions)) if actions[i] == 1]
    total_long_reward = sum(long_rewards) if long_rewards else 0
    total_short_reward = sum(short_rewards) if short_rewards else 0
    avg_long_reward = np.mean(long_rewards) if long_rewards else 0
    avg_short_reward = np.mean(short_rewards) if short_rewards else 0
    
    print(f"### {model_name} Trading Performance\n")
    print(f"#### Long Trades\n")
    print(f"Total Base Profit: ${total_long_profit:.2f}\n")
    print(f"Average Base Profit: ${avg_long_profit:.2f}, {long_trades} trades\n")
    print(f"Total Reward: {total_long_reward:.2f}\n")
    print(f"Average Reward: {avg_long_reward:.2f}\n")
    print(f"Total Trades: {long_trades} trades, {long_trades/len(test_df)*100:.2f}% of test set bars\n")
    profitable_long = sum(1 for p in long_profits_list if p > 0)
    print(f"Profitable Long Trades: {profitable_long}/{long_trades} ({profitable_long/max(long_trades,1)*100:.1f}%)\n")
    
    print(f"#### Short Trades\n")
    print(f"Total Base Profit: ${total_short_profit:.2f}\n")
    print(f"Average Base Profit: ${avg_short_profit:.2f}, {short_trades} trades\n")
    print(f"Total Reward: {total_short_reward:.2f}\n")
    print(f"Average Reward: {avg_short_reward:.2f}\n")
    print(f"Total Trades: {short_trades} trades, {short_trades/len(test_df)*100:.2f}% of test set bars\n")
    profitable_short = sum(1 for p in short_profits_list if p > 0)
    print(f"Profitable Short Trades: {profitable_short}/{short_trades} ({profitable_short/max(short_trades,1)*100:.1f}%)\n")
    
    print(f"#### Overall Trading Statistics\n")
    print(f"Action Distribution: {np.bincount(actions, minlength=3)/len(actions)*100}\n")
    print(f"Total Profit: ${total_long_profit + total_short_profit:.2f}\n")
    print(f"Overall Accuracy: {np.mean(predicted_actions == test_labels)*100:.2f}%\n")
    
    # ===== ADDITIONAL METRICS =====
    # Calculate precision, recall, F1-score for each class
    print(f"#### Precision, Recall, F1-Score by Class\n")
    for class_label in range(3):
        class_name = class_names[class_label]
        tp = confusion_mat[class_label][class_label]  # True positives
        fp = np.sum(confusion_mat[:, class_label]) - tp  # False positives
        fn = np.sum(confusion_mat[class_label, :]) - tp  # False negatives
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"{class_name}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1_score:.3f}\n")
    
    #Return key metrics for comparison
    return {
        'model_name': model_name,
        'overall_accuracy': overall_accuracy,
        'total_profit': total_long_profit + total_short_profit,
        'long_trades': long_trades,
        'short_trades': short_trades,
        'avg_long_profit': avg_long_profit,
        'avg_short_profit': avg_short_profit,
        'profitable_long_rate': profitable_long/max(long_trades,1)*100,
        'profitable_short_rate': profitable_short/max(short_trades,1)*100,
        'confusion_matrix': confusion_mat,
        'results_file': filename,
        'results_dataframe': results_df
    }



In [82]:
"""
Model Evaluation
Evaluates the ensemble model (XGBoost and Random Forest) on the test dataset using price features. Computes predictions, confusion matrix, and trading performance metrics like total profit and trade counts. Logs detailed evaluation results, including accuracy and per-class metrics, to the documentation file. Returns a dictionary with key performance metrics for further analysis.
"""

# Evaluate model
print("Model Evaluation (With Price Features)\n")

results = evaluate_model(xgb_model, rf_model, feature_data_test, test_labels, test_valid_indices, test_df_filtered, "Model")


Model Evaluation (With Price Features)


### Detailed Results Saved

Results saved to: evaluation_results/Model_evaluation_results_20250804_232536.csv

Total records saved: 6575

Signal Distribution in Saved Data:

Actual Signals: {'No Trade': 6400, 'Short': 88, 'Long': 87}

Predicted Signals: {'No Trade': 6575}

Correct Predictions: 6400/6575 (97.34%)

## Model Detailed Evaluation

### Confusion Matrix:

Predicted →

Actual ↓    Long    Short    No Trade

Long (0)      0        0         87    

Short (1)     0        0         88    

No Trade (2)    0        0        6400   


### Detailed Statistics:


Actual Long (0):

Total Cases: 87

Correct Predictions: 0 (0.00%)

Misclassified as No Trade (2): 87 (100.00%)


Actual Short (1):

Total Cases: 88

Correct Predictions: 0 (0.00%)

Misclassified as No Trade (2): 88 (100.00%)


Actual No Trade (2):

Total Cases: 6400

Correct Predictions: 6400 (100.00%)


Overall Statistics:

Total Test Cases: 6575

Total Correct Predictions: 6400 (97

In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import json

def export_trading_data(df, feature_data_test, test_labels, test_valid_indices, test_df, 
                       xgb_model, rf_model, DIRECTORIES, SYMBOL, SUFFIX, VERSION):
    """
    Export trading data to CSV with the exact structure requested:
    1. Raw OHLC data: Datetime, Open, High, Low, Close (for all signal rows including NO_TRADE)
    2. Trade signals based on model predictions: Trade Signal, EntryDateTime, EntryPrice, ExitDateTime, ExitPrice, BaseProfit
    """
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Exporting trading data to CSV...")
    
    # Create output directory if it doesn't exist
    output_dir = Path('./output')
    output_dir.mkdir(exist_ok=True)
    
    # Load asset parameters for proper profit calculation
    with open('user_parameters.json', 'r') as file:
        data = json.load(file)
    asset_data = data['assets'][SYMBOL]
    S_PER_POINT = asset_data['price_per_point']
    MIN_PRICE_RES = asset_data['minimum_resolution_points']
    TARGET_PROFIT = asset_data['targer_profit']
    STOP_LOSS = asset_data['stoploss']
    
    # ===== EXPORT 1: RAW OHLC DATA (FOR ALL SIGNAL ROWS) =====
    print("Exporting raw OHLC data for all signal rows...")
    
    # Get model predictions first
    xgb_probs = xgb_model.predict_proba(feature_data_test)
    rf_probs = rf_model.predict_proba(feature_data_test)
    ensemble_probs = 0.7 * xgb_probs + 0.3 * rf_probs
    
    # Generate predictions
    predicted_actions = np.full(len(feature_data_test), 2)  # Default to No Trade
    
    for i in range(len(ensemble_probs)):
        long_prob, short_prob, no_trade_prob = ensemble_probs[i]
        
        if (long_prob > short_prob and long_prob > no_trade_prob):  
            predicted_actions[i] = 0  # Long
        elif (short_prob >= long_prob and short_prob > no_trade_prob):
            predicted_actions[i] = 1  # Short
    
    # Collect indices for all signal rows (including NO_TRADE)
    all_signal_indices = []
    signal_names = {0: 'LONG', 1: 'SHORT', 2: 'NO_TRADE'}
    
    for i in range(len(feature_data_test)):
        test_idx = test_valid_indices[i] if i < len(test_valid_indices) else 0
        
        # Skip if not enough future data
        if test_idx + 6 >= len(df):
            continue
            
        # Include all signals (LONG, SHORT, NO_TRADE)
        all_signal_indices.append(test_idx)
    
    # Extract OHLC data for all signal rows
    if all_signal_indices:
        ohlc_data = df.iloc[all_signal_indices][['Date', 'Open', 'High', 'Low', 'Close']].copy()
        ohlc_data.columns = ['Datetime', 'Open', 'High', 'Low', 'Close']
        
        # Save raw OHLC data
        ohlc_filename = f'./output/{SYMBOL}_raw_ohlc_data{SUFFIX}{VERSION}.csv'
        ohlc_data.to_csv(ohlc_filename, index=False)
        print(f"Raw OHLC data saved to: {ohlc_filename}")
        print(f"Records: {len(ohlc_data)} (all signal rows including NO_TRADE)")
    else:
        print("No signal rows found - no OHLC data to export")
        ohlc_filename = None
    
    # ===== EXPORT 2: TRADE SIGNALS BASED ON MODEL PREDICTIONS =====
    print("Exporting trade signals based on model predictions...")
    
    # Create trade signals DataFrame
    trade_signals = []
    
    # Process each test prediction
    for i in range(len(feature_data_test)):
        test_idx = test_valid_indices[i] if i < len(test_valid_indices) else 0
        
        # Skip if not enough future data
        if test_idx + 6 >= len(df):
            continue
            
        # Get predicted signal
        predicted_signal = signal_names.get(predicted_actions[i], 'NO_TRADE')
        
        # Entry occurs at the OPEN of the NEXT bar (not current bar)
        entry_datetime = df.iloc[test_idx + 1]['Date']  # Next bar
        entry_price = df.iloc[test_idx + 1]['Open']     # Open of next bar
        
        # Dynamic exit logic - check each bar for stop loss/target profit
        exit_bar = 6  # Default to hard exit at 6th bar
        exit_reason = 'HARD_EXIT'  # Track why the trade exited
        
        if predicted_signal in ['LONG', 'SHORT']:
            # Check for early exits due to stop loss or target profit
            for j in range(1, 7):
                if test_idx + j >= len(df):
                    break
                    
                close_tj = df.iloc[test_idx + j]['Close']
                low_tj = df.iloc[test_idx + j]['Low']
                high_tj = df.iloc[test_idx + j]['High']
                
                if predicted_signal == 'LONG':
                    # Check long position drawdown (stop loss)
                    long_drawdown = ((entry_price - low_tj) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                    if long_drawdown >= STOP_LOSS:
                        exit_bar = j
                        exit_reason = 'STOP_LOSS'
                        break
                    
                    # Check long position profit (target profit)
                    long_profit_at_close = ((close_tj - entry_price) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                    if long_profit_at_close >= TARGET_PROFIT:
                        exit_bar = j
                        exit_reason = 'TARGET_PROFIT'
                        break
                        
                elif predicted_signal == 'SHORT':
                    # Check short position drawdown (stop loss)
                    short_drawdown = ((high_tj - entry_price) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                    if short_drawdown >= STOP_LOSS:
                        exit_bar = j
                        exit_reason = 'STOP_LOSS'
                        break
                    
                    # Check short position profit (target profit)
                    short_profit_at_close = ((entry_price - close_tj) / MIN_PRICE_RES) * (S_PER_POINT * MIN_PRICE_RES)
                    if short_profit_at_close >= TARGET_PROFIT:
                        exit_bar = j
                        exit_reason = 'TARGET_PROFIT'
                        break
        
        # Set exit datetime and price based on determined exit bar
        exit_datetime = df.iloc[test_idx + exit_bar]['Date']
        exit_price = df.iloc[test_idx + exit_bar]['Close']
        
        # Calculate base profit with proper multiplier
        if predicted_signal == 'LONG':
            base_profit = (exit_price - entry_price) * S_PER_POINT
        elif predicted_signal == 'SHORT':
            base_profit = (entry_price - exit_price) * S_PER_POINT
        else:  # NO_TRADE
            base_profit = 0  # No profit for NO_TRADE
            exit_reason = 'NO_TRADE'
            
        trade_signals.append({
            'Trade Signal': predicted_signal,
            'EntryDateTime': entry_datetime,
            'EntryPrice': entry_price,
            'ExitDateTime': exit_datetime,
            'ExitPrice': exit_price,
            'BaseProfit': base_profit,
            'ExitBar': exit_bar,
            'ExitReason': exit_reason
        })
    
    # Create DataFrame and save
    if trade_signals:
        trade_df = pd.DataFrame(trade_signals)
        trade_filename = f'./output/{SYMBOL}_trade_signals{SUFFIX}{VERSION}.csv'
        trade_df.to_csv(trade_filename, index=False)
        print(f"Trade signals saved to: {trade_filename}")
        print(f"Trade records: {len(trade_df)}")
        
        # Print signal distribution
        signal_counts = trade_df['Trade Signal'].value_counts()
        print(f"Signal distribution: {dict(signal_counts)}")
    else:
        print("No trade signals generated")
        trade_filename = None
    
    print(f"\n{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Data export completed!")
    
    return {
        'ohlc_filename': ohlc_filename,
        'trade_filename': trade_filename,
        'total_ohlc_records': len(all_signal_indices) if all_signal_indices else 0,
        'total_trades': len(trade_signals) if trade_signals else 0
    }

if __name__ == "__main__":
    print("Trading data export module loaded successfully!") 

Trading data export module loaded successfully!


In [29]:

print("Starting data export...")

# Export trading data
export_results = export_trading_data(df, feature_data_test, test_labels, test_valid_indices, test_df_filtered, 
                                   xgb_model, rf_model, DIRECTORIES, SYMBOL, SUFFIX, VERSION)

print(f"All exports completed successfully!")
print("Files created:")
print(f"  - Raw OHLC data: {export_results['ohlc_filename']}")
if export_results['trade_filename']:
    print(f"  - Trade signals: {export_results['trade_filename']}")

Starting data export...
2025-08-04 22:04:43: Exporting trading data to CSV...
Exporting raw OHLC data for all signal rows...
Raw OHLC data saved to: ./output/ES_raw_ohlc_data_0825_v10.csv
Records: 6575 (all signal rows including NO_TRADE)
Exporting trade signals based on model predictions...
Trade signals saved to: ./output/ES_trade_signals_0825_v10.csv
Trade records: 6575
Signal distribution: {'NO_TRADE': np.int64(6277), 'LONG': np.int64(213), 'SHORT': np.int64(85)}

2025-08-04 22:04:49: Data export completed!
All exports completed successfully!
Files created:
  - Raw OHLC data: ./output/ES_raw_ohlc_data_0825_v10.csv
  - Trade signals: ./output/ES_trade_signals_0825_v10.csv


In [108]:
pwd

'/Users/hammad/Downloads/ahsan_files_v1'